In [ ]:
%pylab inline --no-import-all 
from pathlib import Path 

In [ ]:
Data_path = Path("../input/geolifeclef-2022-lifeclef-2022-fgvc9")

In [ ]:
ls -L $Data_path

In [ ]:
ls $Data_path/observations

In [ ]:
import pandas as pd 


In [ ]:
df_obs_us = pd.read_csv(Data_path/ "observations"/ "observations_us_train.csv",sep=";",index_col="observation_id")
df_obs_fr = pd.read_csv(Data_path/ "observations"/ "observations_fr_train.csv",sep=";",index_col="observation_id")

df_obs= pd.concat((df_obs_us,df_obs_fr))

print("Number of observation for training: {}".format(len(df_obs)))

df_obs.head()

In [ ]:
df_obs_fr_test= pd.read_csv(Data_path/"observations"/"observations_fr_test.csv",sep=";",index_col="observation_id")
df_obs_us_test= pd.read_csv(Data_path/"observations"/ "observations_us_test.csv",sep=";",index_col="observation_id")

df_obs_test=pd.concat((df_obs_fr_test,df_obs_us_test))

print("Number of observation in test set:{}".format(len(df_obs_test)))

df_obs_test.head()

In [ ]:
!rm -rf GLC
!git clone https://github.com/maximiliense/GLC

In [ ]:
from GLC.plotting import plot_map


def plot_observations_distribution(ax, df_obs, df_obs_test=None, **kwargs):
    default_kwargs = {
        "zorder": 1,
        "alpha": 0.1,
        "s": 0.5
    }
    default_kwargs.update(kwargs)
    kwargs = default_kwargs
    
    ax.scatter(df_obs.longitude, df_obs.latitude, color="blue", **kwargs)
    
    if df_obs_test is not None:
        ax.scatter(df_obs_test.longitude, df_obs_test.latitude, color="red", **kwargs)


fig = plt.figure(figsize=(10, 5.5))
ax = plot_map(region="us")
plot_observations_distribution(ax, df_obs_us, df_obs_us_test)
ax.set_title("Observations distribution (US)")

# fig.savefig("../input/geolifeclef-2022-lifeclef-2022-fgvc9/observations/images/observations_distribution_US.png", bbox_inches="tight", dpi=100, facecolor="white")


fig = plt.figure(figsize=(8, 8))
ax = plot_map(region="fr")
plot_observations_distribution(ax, df_obs_fr, df_obs_fr_test)
ax.set_title("Observations distribution (France)")

# fig.savefig("../input/geolifeclef-2022-lifeclef-2022-fgvc9/observations/observations_distribution_FR.png", bbox_inches="tight", dpi=100, facecolor="white")

In [ ]:
def select_samples_around_point(df_obs,lon_min,lon_max,lat_min,lat_max):
    ind= ((lon_min<=df_obs.longitude) & (lon_max>= df_obs.longitude)
         & (lat_min<= df_obs.latitude) & (lat_max>= df_obs.latitude))
    return df_obs[ind]

extent= [3, 4.5, 43.25, 44.25]

fig= plt.figure(figsize=(9.5,7))
ax= plot_map(extent=extent)

plt.show(ax)

In [ ]:
# df_obs_zoom= select_samples_around_point(df_obs_fr,*extent)
# df_obs_zoom_test= select_samples_around_point(df_obs_fr_test, *extent)

# kwargs = {
#     "alpha": 0.2,
#     "s": 5,
# }

# plot_observations_distribution(ax, df_obs_zoom, df_obs_zoom_test, **kwargs)
# ax.set_title("Observations distribution around Montpellier, France")

df_obs_zoom = select_samples_around_point(df_obs_fr, *extent)
df_obs_zoom_test = select_samples_around_point(df_obs_fr_test, *extent)

kwargs = {
    "alpha": 0.2,
    "s": 5,
}
plot_observations_distribution(ax,df_obs_zoom, df_obs_zoom_test, **kwargs)
ax.set_title("Observations distribution around Montpellier, France")


In [ ]:
def select_samples_around_point(df_obs, lon_min, lon_max, lat_min, lat_max):
    ind = (
        (lon_min <= df_obs.longitude) & (df_obs.longitude <= lon_max)
        & (lat_min <= df_obs.latitude) & (df_obs.latitude <= lat_max)
    )
    return df_obs[ind]


extent = [3, 4.5, 43.25, 44.25]

fig = plt.figure(figsize=(9.5, 7))
ax = plot_map(extent=extent)

df_obs_zoom = select_samples_around_point(df_obs_fr, *extent)
df_obs_zoom_test = select_samples_around_point(df_obs_fr_test, *extent)

kwargs = {
    "alpha": 0.2,
    "s": 5,
}
plot_observations_distribution(ax, df_obs_zoom, df_obs_zoom_test, **kwargs)
ax.set_title("Observations distribution around Montpellier, France")

# fig.savefig("../images/train_test_split.png", bbox_inches="tight", dpi=100, facecolor="white")

In [ ]:
species_value_counts = df_obs["species_id"].value_counts()

print("Total number of species: {}".format(len(species_value_counts)))


fig = plt.figure()
ax = fig.gca()

x = np.arange(len(species_value_counts))
ax.plot(x, species_value_counts)

ax.set_yscale("log")

ax.set_xlabel("ranked species")
ax.set_ylabel("number of observations per species")
ax.set_title("Species observations distribution")

ax.grid()
ax.autoscale(tight=True)
ax.set_ylim(bottom=1)

In [ ]:
df_species= pd.read_csv("../input/geolifeclef-2022-lifeclef-2022-fgvc9/metadata/species_details.csv",sep=";")

print("total_number_of_species:{}".format(len(df_species)))
print("\nNumber of species in each kingdom")
print(df_species.GBIF_kingdom_name.value_counts())

df_species.head()
      

In [ ]:
df_obs= df_obs.reset_index().merge(df_species,on="species_id",how="left").set_index(df_obs.index.names)

print("Number of observation in each kingdom :")
print(df_obs.GBIF_kingdom_name.value_counts())

df_obs.head()

In [ ]:
df_env_vars = pd.read_csv("../input/geolifeclef-2022-lifeclef-2022-fgvc9/metadata/environmental_variables.csv",sep=";")
df_env_vars.head()

In [ ]:
df_landcover_labels = pd.read_csv("../input/geolifeclef-2022-lifeclef-2022-fgvc9/metadata/landcover_original_labels.csv",sep=";")
df_landcover_labels.head()

In [ ]:
df_landcover_suggested_alignment= pd.read_csv("../input/geolifeclef-2022-lifeclef-2022-fgvc9/metadata/landcover_suggested_alignment.csv",sep=";")
df_landcover_suggested_alignment.head()

In [ ]:
ls -L $Data_path

In [ ]:
Patches_path_fr =Data_path /"patches-fr"

Patches_path_us =Data_path /"patches-us"

In [ ]:
ls $Patches_path_fr

In [ ]:
ls $Patches_path_fr/00

In [ ]:
ls $Patches_path_fr/44/14/10171444*

In [ ]:
ls $Patches_path_us/00/81/22068100*

In [ ]:
print(Patches_path_fr)

In [ ]:
from GLC.data_loading.common import load_patch
patch = load_patch(10171444,Data_path)

print("Number of data sources: {}".format(len(patch)))
print("Array shape: {}".format([p.shape for p in patch]))
print("Data type:{}".format([p.dtype for p in patch]))

In [ ]:
landcover_mapping= df_landcover_suggested_alignment["suggested_landcover_code"].values

patch= load_patch(1071444,Data_path,landcover_mapping=landcover_mapping)

In [ ]:
from GLC.plotting import visualize_observation_patch

landcover_mapping= df_landcover_suggested_alignment[["suggested_landcover_code","suggested_landcover_label"]].drop_duplicates().sort_values("suggested_landcover_code")["suggested_landcover_label"].values

fig= visualize_observation_patch(patch,observation_data=df_obs.loc[10171444], landcover_labels=landcover_labels, return_fig=True)